In [2]:
import numpy as np

In [3]:
k = np.fromfile('../data/k.bin', dtype=np.float32).reshape(-1, 128)
q = np.fromfile('../data/q.bin', dtype=np.float32).reshape(-1, 128)
(k.shape, q.shape)

((128000, 128), (128000, 128))

In [7]:
k_l2 = np.linalg.norm(k, axis=1)
q_l2 = np.linalg.norm(q, axis=1)
print(k_l2.mean(), k_l2.std())
print(q_l2.mean(), q_l2.std())

23.511227 2.3082576
26.730055 2.0312736


In [ ]:
k_norm = k / k_l2[:, None]
q_norm = q / q_l2[:, None]
k_norm.tofile('../data/k_norm.bin')
q_norm.tofile('../data/q_norm.bin')

In [ ]:
R = 64
k_aug = np.concatenate([k, np.sqrt(R**2 - k_l2**2)[:, None]], axis=1)
q_aug = np.concatenate([q, np.zeros((q.shape[0], 1))], axis=1)
k_aug.tofile('../data/k_aug.bin')
q_aug.tofile('../data/q_aug.bin')

64.0 1.8832351e-06
